In [4]:
import numpy as np
from numpy import shape

from datetime import date

import random
import math
import dill
import glob
import gc

import matplotlib.pyplot as plt

from random import seed

# Import train_test_split function
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import device

# Tensorflow soll auf CPU und nicht auf der GPU laufen
device("cpu:0")

# für GPU:
# tf.device("gpu:0")

from tensorflow.keras.utils import to_categorical
from keras.layers import Dense
from keras.models import Sequential # Documentation: https://keras.io/models/sequential/

In [6]:
%run '../../_functions/functions.py'

### Load MNIST images

In [7]:
# # MNIST von sklearn mit 28x28 Pixel
# from sklearn.datasets import fetch_openml

# # Load data from https://www.openml.org/d/554
# X_raw, y_raw = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False)

In [8]:
# Import data that was saved on this system

raw_data = load_file(file = "../raw_data/MNIST_images.pkl")
X_raw = raw_data[0]
y_raw = raw_data[1]

In [9]:
# Filter on indices of '0' and '8'
indices = (y_raw == '0') | (y_raw == '8')

# Reshape into squares
X_raw = X_raw[indices, :]
X = X_raw.reshape([sum(indices), 28, 28])

y = y_raw[indices]

# Factorize into dichotomous variable
Y = pd.factorize(y)[0]

In [10]:
# We analyse 1998 samples
y = y[0:1998]
Y = Y[0:1998]

np.unique(y, return_counts=True)

(array(['0', '8'], dtype=object), array([1032,  966]))

### Load Persistences Landscapes

In [11]:
pl_folderpath = "../persistence_landscapes_averages/"

In [12]:
# # Shows filenames of loaded files
# print(glob.glob(pl_folderpath + "avgPL_bucket01_H0_scaledWithin*.pkl")[-1])
# print(glob.glob(pl_folderpath + "avgPL_bucket01_H1_scaledWithin*.pkl")[-1])
# print(glob.glob(pl_folderpath + "avgPL_bucket01_H0_scaledBetween*.pkl")[-1])
# print(glob.glob(pl_folderpath + "avgPL_bucket01_H1_scaledBetween*.pkl")[-1])
# print(glob.glob(pl_folderpath + "avgPL_bucket01_H0_unscaled*.pkl")[-1])
# print(glob.glob(pl_folderpath + "avgPL_bucket01_H1_unscaled*.pkl")[-1])

In [13]:
avgPL_bucket01_H0_scaledWithin = load_file(file = glob.glob(pl_folderpath + "avgPL_bucket01_H0_scaledWithin*.pkl")[-1])
avgPL_bucket01_H1_scaledWithin = load_file(file = glob.glob(pl_folderpath + "avgPL_bucket01_H1_scaledWithin*.pkl")[-1])

avgPL_bucket01_H0_scaledBetween = load_file(file = glob.glob(pl_folderpath + "avgPL_bucket01_H0_scaledBetween*.pkl")[-1])
avgPL_bucket01_H1_scaledBetween = load_file(file = glob.glob(pl_folderpath + "avgPL_bucket01_H1_scaledBetween*.pkl")[-1])

avgPL_bucket01_H0_unscaled = load_file(file = glob.glob(pl_folderpath + "avgPL_bucket01_H0_unscaled*.pkl")[-1])
avgPL_bucket01_H1_unscaled = load_file(file = glob.glob(pl_folderpath + "avgPL_bucket01_H1_unscaled*.pkl")[-1])

# Neural Networks on Raw Data

In [14]:
no_relevant_samples = len(avgPL_bucket01_H1_unscaled)
no_relevant_samples

1998

In [15]:
# Folderpath to store the results
folderpath_results = "../results/"

In [16]:
# Parameter for persistence landscapes
pl_resolution = 250
pl_num_landscapes = 10

### Train-Test-Splits

In [17]:
# Train-test split with data in vector format
image_vector_size = 28*28

# Transform into categorical variable
Y_categorical = to_categorical(Y)

X_raw_train, X_raw_test, y_raw_train, y_raw_test = train_test_split(X[0:no_relevant_samples].reshape(no_relevant_samples, image_vector_size),
                                                                    np.asarray(Y),
                                                                    test_size = 0.3,
                                                                    random_state = 2)

In [18]:
def simple_NN_rawdata_model_1():
    
    model = Sequential()
    model.add(Dense(32, activation='relu', input_shape=[28*28,]))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
        
    return model

def simple_NN_rawdata_model_2():
    
    model = Sequential()
    model.add(Dense(16, activation='relu', input_shape=[28*28,]))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
        
    return model

def simple_NN_rawdata_model_3():
    
    model = Sequential()
    model.add(Dense(8, activation='relu', input_shape=[28*28,]))
    model.add(Dense(4, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
        
    return model

In [19]:
# Hyperparameters for gridsearch
simple_NN_rawdata_model = [simple_NN_rawdata_model_1, simple_NN_rawdata_model_2, simple_NN_rawdata_model_3]
simple_NN_rawdata_learning_rates = [1e-2, 1e-3, 1e-4, 1e-5, 1e-6]
simple_NN_rawdata_validation_splits = [0.33, None]
simple_NN_rawdata_epochs = [3, 4, 5, 8, 10, 20, 50]

In [20]:
grid_simple_NN_rawdata = create_modelgrid(models = simple_NN_rawdata_model,
                                          learning_rates = simple_NN_rawdata_learning_rates,
                                          validation_splits = simple_NN_rawdata_validation_splits,
                                          epochs = simple_NN_rawdata_epochs)

In [21]:
results_simple_NN_rawdata = test_multiple_models(x_train = X_raw_train,
                                                 y_train = y_raw_train,
                                                 x_test = X_raw_test,
                                                 y_test = y_raw_test,
                                                 modelgrid = grid_simple_NN_rawdata)

Training model 1 of 210


2022-09-07 08:31:14.704074: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Training model 2 of 210
Training model 3 of 210
Training model 4 of 210
Training model 5 of 210
Training model 6 of 210
Training model 7 of 210
Training model 8 of 210
Training model 9 of 210
Training model 10 of 210
Training model 11 of 210
Training model 12 of 210
Training model 13 of 210
Training model 14 of 210
Training model 15 of 210
Training model 16 of 210
Training model 17 of 210
Training model 18 of 210
Training model 19 of 210
Training model 20 of 210
Training model 21 of 210
Training model 22 of 210
Training model 23 of 210
Training model 24 of 210
Training model 25 of 210
Training model 26 of 210
Training model 27 of 210
Training model 28 of 210
Training model 29 of 210
Training model 30 of 210
Training model 31 of 210
Training model 32 of 210
Training model 33 of 210
Training model 34 of 210
Training model 35 of 210
Training model 36 of 210
Training model 37 of 210
Training model 38 of 210
Training model 39 of 210
Training model 40 of 210
Training model 41 of 210
Training

In [22]:
results_simple_NN_rawdata = results_simple_NN_rawdata.sort_values(by='accuracy', ascending=False).reset_index(drop=True)

# Save results to csv
results_simple_NN_rawdata.to_csv(folderpath_results + "results_simple_NN_rawdata.csv",
                                              encoding = 'utf-8',
                                              index = False)

results_simple_NN_rawdata

,model,learning_rate,val_split,epochs,accuracy,TPR,TNR
0,simple_NN_rawdata_model_3,0.001000,None,50,0.991667,0.989399,0.993691
1,simple_NN_rawdata_model_1,0.010000,None,50,0.991667,0.992933,0.990536
2,simple_NN_rawdata_model_1,0.010000,None,8,0.990000,0.996466,0.984227
3,simple_NN_rawdata_model_1,0.010000,None,10,0.990000,0.992933,0.987382
4,simple_NN_rawdata_model_1,0.010000,0.33,5,0.988333,0.989399,0.987382
...,...,...,...,...,...,...,...
205,simple_NN_rawdata_model_3,0.000001,0.33,20,0.395000,0.473498,0.324921
206,simple_NN_rawdata_model_3,0.000001,None,10,0.395000,0.455830,0.340694
207,simple_NN_rawdata_model_1,0.000001,0.33,5,0.340000,0.540636,0.160883
208,simple_NN_rawdata_model_3,0.000010,0.33,4,0.336667,0.537102,0.157729


# Neural Networks on Persistence Landscapes

In [23]:
# H1 scaledBetween
seed(999)
# Train-Test-Split 
X_b01_scaledBetween_H1_train, X_b01_scaledBetween_H1_test, Y_b01_scaledBetween_H1_train, Y_b01_scaledBetween_H1_test = train_test_split(avgPL_bucket01_H1_scaledBetween,
                                                                                                                                        Y,
                                                                                                                                        test_size=0.2)

# H1 scaledWithin                                                                                                                                 test_size=0.2)
seed(999)
# Train-Test-Split 
X_b01_scaledWithin_H1_train, X_b01_scaledWithin_H1_test, Y_b01_scaledWithin_H1_train, Y_b01_scaledWithin_H1_test = train_test_split(avgPL_bucket01_H1_scaledWithin,
                                                                                                                                    Y,
                                                                                                                                    test_size=0.2)

# H1 Unscaled                                                                                                       test_size=0.2)
seed(999)
# Train-Test-Split 
X_b01_unscaled_H1_train, X_b01_unscaled_H1_test, Y_b01_unscaled_H1_train, Y_b01_unscaled_H1_test = train_test_split(avgPL_bucket01_H1_unscaled,
                                                                                                                    Y,
                                                                                                                    test_size=0.2)

### NN Models for Persistence Landscapes

In [24]:
def simple_NN_PL_model_1():
    
    model = Sequential()
    model.add(Dense(32, activation='relu', input_shape=[2500,]))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
        
    return model

def simple_NN_PL_model_2():
    
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=[2500,]))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
        
    return model

def simple_NN_PL_model_3():
    
    model = Sequential()
    model.add(Dense(128, activation='relu', input_shape=[2500,]))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    return model

In [25]:
simple_NN_PL_models = [simple_NN_PL_model_1, simple_NN_PL_model_2, simple_NN_PL_model_3]
simple_NN_PL_learning_rates = [1e-2, 1e-3, 1e-4, 1e-5, 1e-6]
simple_NN_PL_validation_splits = [0.33, None]
simple_NN_PL_epochs = [3, 5, 8, 10, 20, 50]

### Train Models on Persistence Landscapes (scaledBetween)

In [26]:
grid_simple_NN_pl_scaledBetween = create_modelgrid(models = simple_NN_PL_models,
                                                   learning_rates = simple_NN_PL_learning_rates,
                                                   validation_splits = simple_NN_PL_validation_splits,
                                                   epochs = simple_NN_PL_epochs)
grid_simple_NN_pl_scaledBetween.head(10)

,model,models_to_combine,learning_rate,validation_split,epochs
0,<function simple_NN_PL_model_1 at 0x7f34dc260790>,None,0.01,0.33,3
1,<function simple_NN_PL_model_2 at 0x7f2f6d120e50>,None,0.01,0.33,3
2,<function simple_NN_PL_model_3 at 0x7f2f6d120940>,None,0.01,0.33,3
3,<function simple_NN_PL_model_1 at 0x7f34dc260790>,None,0.001,0.33,3
4,<function simple_NN_PL_model_2 at 0x7f2f6d120e50>,None,0.001,0.33,3
5,<function simple_NN_PL_model_3 at 0x7f2f6d120940>,None,0.001,0.33,3
6,<function simple_NN_PL_model_1 at 0x7f34dc260790>,None,0.0001,0.33,3
7,<function simple_NN_PL_model_2 at 0x7f2f6d120e50>,None,0.0001,0.33,3
8,<function simple_NN_PL_model_3 at 0x7f2f6d120940>,None,0.0001,0.33,3
9,<function simple_NN_PL_model_1 at 0x7f34dc260790>,None,0.00001,0.33,3


In [27]:
results_simple_NN_b01_scaledBetween_H1 = test_multiple_models(x_train = np.asarray(X_b01_scaledBetween_H1_train),
                                                              y_train = np.asarray(Y_b01_scaledBetween_H1_train),
                                                              x_test = np.asarray(X_b01_scaledBetween_H1_test),
                                                              y_test = np.asarray(Y_b01_scaledBetween_H1_test),
                                                              modelgrid = grid_simple_NN_pl_scaledBetween)

Training model 1 of 180
Training model 2 of 180
Training model 3 of 180
Training model 4 of 180
Training model 5 of 180
Training model 6 of 180
Training model 7 of 180
Training model 8 of 180
Training model 9 of 180
Training model 10 of 180
Training model 11 of 180
Training model 12 of 180
Training model 13 of 180
Training model 14 of 180
Training model 15 of 180
Training model 16 of 180
Training model 17 of 180
Training model 18 of 180
Training model 19 of 180
Training model 20 of 180
Training model 21 of 180
Training model 22 of 180
Training model 23 of 180
Training model 24 of 180
Training model 25 of 180
Training model 26 of 180
Training model 27 of 180
Training model 28 of 180
Training model 29 of 180
Training model 30 of 180
Training model 31 of 180
Training model 32 of 180
Training model 33 of 180
Training model 34 of 180
Training model 35 of 180
Training model 36 of 180
Training model 37 of 180
Training model 38 of 180
Training model 39 of 180
Training model 40 of 180
Training 

In [28]:
results_simple_NN_b01_scaledBetween_H1 = results_simple_NN_b01_scaledBetween_H1.sort_values(by='accuracy', ascending=False).reset_index(drop=True)

# Save results to csv
results_simple_NN_b01_scaledBetween_H1.to_csv(folderpath_results + "results_simple_NN_b01_scaledBetween_H1.csv",
                                              encoding='utf-8',
                                              index=False)

# Print results
results_simple_NN_b01_scaledBetween_H1

,model,learning_rate,val_split,epochs,accuracy,TPR,TNR
0,simple_NN_PL_model_1,0.000001,0.33,50,0.5725,0.338710,0.775701
1,simple_NN_PL_model_2,0.000001,0.33,50,0.5700,0.704301,0.453271
2,simple_NN_PL_model_1,0.000100,0.33,8,0.5625,0.586022,0.542056
3,simple_NN_PL_model_2,0.000100,0.33,5,0.5575,0.397849,0.696262
4,simple_NN_PL_model_3,0.010000,0.33,20,0.5550,0.284946,0.789720
...,...,...,...,...,...,...,...
175,simple_NN_PL_model_3,0.000001,None,20,0.4475,0.354839,0.528037
176,simple_NN_PL_model_2,0.000001,0.33,10,0.4450,0.376344,0.504673
177,simple_NN_PL_model_1,0.000010,0.33,8,0.4400,0.655914,0.252336
178,simple_NN_PL_model_1,0.000001,None,20,0.4400,0.489247,0.397196


### Train Models on Persistence Landscapes for H1 (scaledWithin)

In [29]:
grid_simple_NN_pl_scaledWithin = create_modelgrid(models = simple_NN_PL_models,
                                                   learning_rates = simple_NN_PL_learning_rates,
                                                   validation_splits = simple_NN_PL_validation_splits,
                                                   epochs = simple_NN_PL_epochs)
grid_simple_NN_pl_scaledWithin.head()

,model,models_to_combine,learning_rate,validation_split,epochs
0,<function simple_NN_PL_model_1 at 0x7f34dc260790>,None,0.01,0.33,3
1,<function simple_NN_PL_model_2 at 0x7f2f6d120e50>,None,0.01,0.33,3
2,<function simple_NN_PL_model_3 at 0x7f2f6d120940>,None,0.01,0.33,3
3,<function simple_NN_PL_model_1 at 0x7f34dc260790>,None,0.001,0.33,3
4,<function simple_NN_PL_model_2 at 0x7f2f6d120e50>,None,0.001,0.33,3


In [30]:
results_simple_NN_b01_scaledWithin_H1 = test_multiple_models(x_train = np.asarray(X_b01_scaledWithin_H1_train),
                                                              y_train = np.asarray(Y_b01_scaledWithin_H1_train),
                                                              x_test = np.asarray(X_b01_scaledWithin_H1_test),
                                                              y_test = np.asarray(Y_b01_scaledWithin_H1_test),
                                                              modelgrid = grid_simple_NN_pl_scaledWithin)

Training model 1 of 180
Training model 2 of 180
Training model 3 of 180
Training model 4 of 180
Training model 5 of 180
Training model 6 of 180
Training model 7 of 180
Training model 8 of 180
Training model 9 of 180
Training model 10 of 180
Training model 11 of 180
Training model 12 of 180
Training model 13 of 180
Training model 14 of 180
Training model 15 of 180
Training model 16 of 180
Training model 17 of 180
Training model 18 of 180
Training model 19 of 180
Training model 20 of 180
Training model 21 of 180
Training model 22 of 180
Training model 23 of 180
Training model 24 of 180
Training model 25 of 180
Training model 26 of 180
Training model 27 of 180
Training model 28 of 180
Training model 29 of 180
Training model 30 of 180
Training model 31 of 180
Training model 32 of 180
Training model 33 of 180
Training model 34 of 180
Training model 35 of 180
Training model 36 of 180
Training model 37 of 180
Training model 38 of 180
Training model 39 of 180
Training model 40 of 180
Training 

In [31]:
results_simple_NN_b01_scaledWithin_H1 = results_simple_NN_b01_scaledWithin_H1.sort_values(by='accuracy', ascending=False).reset_index(drop=True)

# Save results to csv
results_simple_NN_b01_scaledWithin_H1.to_csv(folderpath_results + "results_simple_NN_b01_scaledWithin_H1.csv",
                                              encoding='utf-8',
                                              index=False)

# Print results
results_simple_NN_b01_scaledWithin_H1

,model,learning_rate,val_split,epochs,accuracy,TPR,TNR
0,simple_NN_PL_model_3,0.000010,None,3,0.5475,0.409091,0.683168
1,simple_NN_PL_model_3,0.001000,0.33,20,0.5475,0.282828,0.806931
2,simple_NN_PL_model_3,0.001000,0.33,10,0.5450,0.580808,0.509901
3,simple_NN_PL_model_3,0.010000,None,10,0.5450,0.520202,0.569307
4,simple_NN_PL_model_3,0.000100,0.33,10,0.5425,0.318182,0.762376
...,...,...,...,...,...,...,...
175,simple_NN_PL_model_3,0.000010,None,10,0.4550,0.449495,0.460396
176,simple_NN_PL_model_2,0.000100,None,5,0.4500,0.308081,0.589109
177,simple_NN_PL_model_2,0.000001,0.33,50,0.4475,0.580808,0.316832
178,simple_NN_PL_model_1,0.000001,None,3,0.4400,0.590909,0.292079


### Train Models on Persistence Landscapes for H1 (unscaled)

In [32]:
grid_simple_NN_pl_unscaled = create_modelgrid(models = simple_NN_PL_models,
                                              learning_rates = simple_NN_PL_learning_rates,
                                              validation_splits = simple_NN_PL_validation_splits,
                                              epochs = simple_NN_PL_epochs)
grid_simple_NN_pl_unscaled.head()

,model,models_to_combine,learning_rate,validation_split,epochs
0,<function simple_NN_PL_model_1 at 0x7f34dc260790>,None,0.01,0.33,3
1,<function simple_NN_PL_model_2 at 0x7f2f6d120e50>,None,0.01,0.33,3
2,<function simple_NN_PL_model_3 at 0x7f2f6d120940>,None,0.01,0.33,3
3,<function simple_NN_PL_model_1 at 0x7f34dc260790>,None,0.001,0.33,3
4,<function simple_NN_PL_model_2 at 0x7f2f6d120e50>,None,0.001,0.33,3


In [33]:
results_simple_NN_b01_unscaled_H1 = test_multiple_models(x_train = np.asarray(X_b01_unscaled_H1_train),
                                                             y_train = np.asarray(Y_b01_unscaled_H1_train),
                                                             x_test = np.asarray(X_b01_unscaled_H1_test),
                                                             y_test = np.asarray(Y_b01_unscaled_H1_test),
                                                             modelgrid = grid_simple_NN_pl_unscaled)

Training model 1 of 180
Training model 2 of 180
Training model 3 of 180
Training model 4 of 180
Training model 5 of 180
Training model 6 of 180
Training model 7 of 180
Training model 8 of 180
Training model 9 of 180
Training model 10 of 180
Training model 11 of 180
Training model 12 of 180
Training model 13 of 180
Training model 14 of 180
Training model 15 of 180
Training model 16 of 180
Training model 17 of 180
Training model 18 of 180
Training model 19 of 180
Training model 20 of 180
Training model 21 of 180
Training model 22 of 180
Training model 23 of 180
Training model 24 of 180
Training model 25 of 180
Training model 26 of 180
Training model 27 of 180
Training model 28 of 180
Training model 29 of 180
Training model 30 of 180
Training model 31 of 180
Training model 32 of 180
Training model 33 of 180
Training model 34 of 180
Training model 35 of 180
Training model 36 of 180
Training model 37 of 180
Training model 38 of 180
Training model 39 of 180
Training model 40 of 180
Training 

In [34]:
results_simple_NN_b01_unscaled_H1 = results_simple_NN_b01_unscaled_H1.sort_values(by='accuracy', ascending=False).reset_index(drop=True)

# Save results to csv
results_simple_NN_b01_unscaled_H1.to_csv(folderpath_results + "results_simple_NN_b01_unscaled_H1.csv",
                                              encoding='utf-8',
                                              index=False)

results_simple_NN_b01_unscaled_H1

,model,learning_rate,val_split,epochs,accuracy,TPR,TNR
0,simple_NN_PL_model_3,0.010000,None,50,0.5825,0.406091,0.753695
1,simple_NN_PL_model_3,0.000100,None,8,0.5800,0.380711,0.773399
2,simple_NN_PL_model_2,0.000100,None,3,0.5775,0.558376,0.596059
3,simple_NN_PL_model_3,0.000100,0.33,20,0.5750,0.771574,0.384236
4,simple_NN_PL_model_3,0.010000,None,5,0.5750,0.360406,0.783251
...,...,...,...,...,...,...,...
175,simple_NN_PL_model_3,0.000001,0.33,5,0.4725,0.538071,0.408867
176,simple_NN_PL_model_2,0.000010,0.33,10,0.4650,0.360406,0.566502
177,simple_NN_PL_model_1,0.000001,0.33,20,0.4550,0.456853,0.453202
178,simple_NN_PL_model_2,0.000010,0.33,5,0.4525,0.507614,0.399015
